In [1]:
# import numpy as np
# import random


# class GradientDescent():
    
#     """Descent gradient class with regularize technique
    
#     Parameters
#     ----------
#     regularize : bool
#         If True, the regularization is used.
#     bias : bool
#         If the True, a bias is added to the features.
#     alpha : float > 0
#         Coefficient for the step when updating the parameters.
    
#     Notes
#     -----
#     This class aims at computing the parameters of a linear model using
#     a descent gradient method with or without regularization.
#     """
    
#     def __init__(self, regularize=False, bias=True, alpha=3e-9, iterations=30000):
#         self.bias = bias
#         if alpha < 0:
#             raise ValueError('Alpha parameter must be > 0. Here {}.'.format(alpha))
#         self.alpha = alpha
#         self.iterations = iterations
#         self.regularize = regularize
        
#         #set the epsilon value depending on the regularize case
#         if regularize:
#             self.epsilon = 1e-10
#         else:
#             self.epsilon = 1e-8
    
#     def predict(self, new_features):
#         """Make predictions using the result of the gradient descent
        
#         Parameters
#         ----------
#         new_features : 2d sequence of float
#             The feature for which to predict the labels.
            
#         Returns
#         -------
#         predicted_labels : 2d sequence of float
#             The predicted labels
        
#         Notes
#         -----
#         The method fit must be called first.
#         """
        
#         if self.bias:
#             new_features = self._add_bias(new_features)
#         return self.hypothesis(new_features, self.parameters_)
    
    
#     def fit(self, features, label, parameters=None):
#         """Find the optimal parameters
        
#         Parameters
#         ----------
#         features : 2d sequence of float
#             The input parameters.
#         label : 2d sequence of float
#             The output parameters
#         parameters : 2d sequence of float
#             The initial guess for the descent gradient.
#         """
#         # add bias or not
#         if self.bias:
#             features = self._add_bias(features)
        
#         # if no initial parameters are given get some randomly
#         if parameters is None:
#             n = features.shape[1]
#             parameters = np.random.rand(n,1)
    
#         # compute the initial prediction
#         predictions = self.hypothesis(features, parameters)
        
#         # solve depending of the regularization or not
#         self.parameters_ = self._regularize_fit(features, label, parameters, predictions)

    
#     def _regularize_fit(self, features, label, parameters, predictions):
#         """Find the optimal parameters with regularized method
#         """

#         m = features.shape[0]
        
#         lmb1 = 0.2
#         if self.regularize == False:
#             lmb1 = 0
#         lmb2 = 0
        
#         if self.regularize == 'rigde':
#             lmb2 = 0
#         if self.regularize == 'lasso':
#             lmb2 = 1
#         if self.regularize == 'elastic-net':
#             lmb2 = 0.4
#         print('lambda 1 est ',lmb1)
#         print('lambda 2 est ',lmb2)
        
        
#         costFct = 0
#         costFctEvol = []
#         count = 0
#         while self.testRegCostFct(predictions, label, lmb1, lmb2, parameters, costFct, self.epsilon):
#             count += 1
#             costFct = self.regCostFunction(predictions, label, lmb1, lmb2, parameters)
#             grads = self.regGradients(predictions, label, features, lmb1, lmb2, parameters)
#             parameters = self.updateParameters(parameters, grads, self.alpha)
#             predictions = self.hypothesis(features, parameters)
#             costFctEvol.append(costFct)
            
#         plt.xlabel('Iterations')
#         plt.ylabel('Fonction cout')
#         plt.title('Evolution de la fonction cout en fonctions des iterations')
#         plt.plot(costFctEvol)
#         return parameters
    
#     def _add_bias(self, features):
#         """Add bias column (1 vector)
#         """
#         bias = np.ones(features.shape[0])
#         return np.column_stack([features, bias])
        
#     def hypothesis(self, x, theta):
#         """Compute our hypothesis model (linear regression), use a fonction:
#         """
#         return np.dot(x, theta)
    
#     def costFunction(self, yhat, y):
#         """Fonction de coût
#         """
#         return np.square(yhat - y).sum() / (2*y.shape[0])
    
#     def regCostFunction(self, yhat, y, lmb1, lmb2, theta):
#         """Fonction de coût régularisée
#         """
#         #return self.costFunction(yhat, y) + lmb1/((2*y.shape[0]) * np.square(theta)).sum()


#         return self.costFunction(yhat, y) + lmb1*(((1-lmb2)/2) * np.square(theta).sum() + lmb2*(np.abs(theta)).sum())
    
#     def gradients(self, yhat, y, x):
#         """Dérivée de la fonction de coût == gradients
#         """
        
#         return (((yhat - y) * x).sum(axis=0) / x.shape[0]).reshape(x.shape[1],1)

#     def regGradients(self, yhat, y, x, lmb1 ,lmb2 , theta):
#         """Dérivée de la fonction de coût regularisée
#         """
#         #return (((yhat - y) * x).sum(axis=0) / x.shape[0]).reshape(x.shape[1],1) + lmb1/x.shape[0]*theta

#         return (((yhat - y) * x).sum(axis=0) / x.shape[0]).reshape(x.shape[1],1) + (lmb1*(1-lmb2))/x.shape[0]*theta
    
#     def updateParameters(self, parameters, grads, alpha):
#         """Gradient descent: mise à jour des paramètres
#         """
#         return parameters - alpha * grads
    
#     def testRegCostFct(self, yhat, y, lmb1, lmb2, theta, prevCostFct, epsilon):
#         """ Fonction pour tester l'évolution de la fonction de coût régularisée
            
#             Returns
#             -------
#             test : bool
#                 vrai = continuer la descente de gradient
#         """
#         return np.abs(self.regCostFunction(yhat, y, lmb1, lmb2, theta) - prevCostFct) >= epsilon*prevCostFct
    
#     def train_test_split(self, X, y, ratio=0.3, random_seed = 42):
#         """ Fonction pour subdiviser les donnees en donnees d'entrainement et donnees de test.
        
#              Parametres
#             ----------
#             X : 2d sequence of float
#                 The input parameters.
#             y : 2d sequence of float
#                 The output parameters
#             ratio : La ratio du test set.
            
#             Returns
#             -------
#             X_train : les features d'entrainement.
#             y_train : les labels d'entrainement.
#             X_test : les features de test.
#             y_test : les labels de test.
            
#         """
#         X_train = []
#         y_train = []
#         X_test = []
#         y_test = []
#         rows = len(X)
#         random.seed(random_seed)
#         test_index = random.sample(range(0,rows), int(rows*ratio))
#         for i in range(rows):
#             if i in test_index:
#                 X_test.append(X[i])
#                 y_test.append(y[i])
#             else:
#                 X_train.append(X[i])
#                 y_train.append(y[i])
#         return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)
    
#     def mae(self, y, y_pred) :
#         return np.abs(y_pred - y).sum() / (2*y.shape[0])
    
#     def rmse(self, y, y_pred) :
#         return np.sqrt(np.square(y_pred - y).sum() / (2*y.shape[0]))
    
#     def r2_score(self, y, y_pred) :
#         return 1 - (np.square(y_pred - y).sum() / np.square(y_pred - np.mean(y)).sum())
        

In [2]:
import gradient_descent

In [3]:
from gradient_descent import GradientDescent

In [4]:
g = GradientDescent()

In [5]:
best[1]

NameError: name 'best' is not defined

In [ ]:
g = GradientDescent(regularize = 'ridge',learning_rate = 0.03, normalize = False)

In [ ]:
import numpy as np 
import pandas as pd
np.random.seed(67)
X = 3 * np.random.rand(100, 1) - 2
Y = 2*X + 7 + np.random.randn(100, 1)

In [ ]:
# from sklearn.datasets import load_diabetes
# diabetes = load_diabetes()
# X = diabetes.data
# Y = diabetes.target

In [ ]:
X.shape

In [ ]:
1 > 9e100

In [ ]:
Y.shape

In [ ]:
Y = Y.reshape(-1,1)
Y.shape

In [ ]:
len(np.std(X, axis = 0))

In [ ]:
X_train, y_train, X_test, y_test = g.train_test_split(X,Y,ratio=0.3,random_seed=44)

In [ ]:
X_train.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.figure()
# plt.scatter(X_train,y_train,alpha = 0.7)
# plt.xlabel('Features')
# plt.ylabel('Labels')

In [ ]:
np.arange(0,1,0.1)

In [ ]:
s = g.tunning(X_train, y_train, "mae", nb_params=10, validation_ratio = 0.3)

In [ ]:
maxi = 0
best = 0
for i in lambd:
    g.fit(X_train,y_train, lambd = i)
    y_pred = g.predict(X_test)
    score = g.rmse(y_pred, y_test)
    if score > maxi:
        maxi = score
    best = i

In [ ]:
best

In [ ]:
lambd

In [ ]:
slope , intercept = g.parameters_[0][0],  g.parameters_[1][0]

In [ ]:
g.parameters_

In [ ]:
slope

In [ ]:
# plt.figure()
# plt.scatter(X,Y,alpha = 0.7)
# plt.plot(X,X*slope+intercept, color = 'red')
# plt.xlabel('Features')
# plt.ylabel('Labels')

In [ ]:
y_pred = g.predict(X_test)

In [ ]:
g.r2_score(y_pred, y_test)